# content-aggregation-on-OER-objects

In [1]:
import sys
#!{sys.executable} -m pip install pikepdf
import pikepdf
import urllib.request
import pandas as pd


create temp folder if not already created

In [2]:
from pathlib import Path
Path("temp").mkdir(parents=True, exist_ok=True)

Input an OER Adress

example: ```https://bildungsportal.sachsen.de/opal/oer/10SYjW2AWSFYY```

In [3]:
pdf_url = input("What is the URL of the pdf? ")

What is the URL of the pdf? https://bildungsportal.sachsen.de/opal/oer/10SYjW2AWSFYY


download and open the pdf with pikepdf

create a dictionary for the Metadata

In [57]:
pdf_filename, headers = urllib.request.urlretrieve(pdf_url, "temp/temp.pdf")
pdf = pikepdf.Pdf.open(pdf_filename)
docinfo = pdf.docinfo
metadict = {}

write all informations from docinfo into metadict (old code)

In [14]:
for key, value in docinfo.items():  
    metadict.update({key:str(value)})

 Autoreninformation (Name✔, Herkunft✔)
 
 Datum der Erstellung✔
 
 Titel✔
 
 Seitenzahl✔
 
 Format✔ (Dokument, Präsentation)

In [84]:
from datetime import datetime

try:
    docinfo["/Author"]
except KeyError:
    metadict.update({"Author":""})
else:
    metadict.update({"Author":str(docinfo["/Author"])})

try:
    docinfo["/Publisher"]
except KeyError:
    metadict.update({"Publisher":""})
else:
    metadict.update({"Publisher":str(docinfo["/Publisher"])})

try:
    docinfo["/CreationDate"]
except KeyError:
    metadict.update({"Datum der Erstellung":""})
else:
    creation = str(docinfo["/CreationDate"])
    date = datetime.strptime(creation.replace("'", ""), "D:%Y%m%d%H%M%S%z")
    metadict.update({"Datum der Erstellung":date.strftime("%x")})
    
try:
    docinfo["/Title"]
except KeyError:
    metadict.update({"Titel":""})
else:
    metadict.update({"Titel":str(docinfo["/Title"])})
    
metadict.update({"Seitenzahl":len(pdf.pages)})

if (pdf.pages[0].mediabox[2] - pdf.pages[0].mediabox[0]) > (pdf.pages[0].mediabox[3] - pdf.pages[0].mediabox[1]):
    metadict.update({"Format":"Dokument"})
else:
    metadict.update({"Format":"Präsentation"})

In [85]:
s = pd.Series(metadict)
s.to_csv("temp/meta.csv")
print("Saved to temp/meta.csv")
pdf.close() 

Saved to temp/meta.csv


In [ ]:
import os
os.remove(pdf_filename)